# Fasta genome reorganization

The goal is to reorganize the genome of the bacteria where there are visible misassemblies on the HicMap. Bacteria concerned by these rearrangements are: 

|Bacteria|Number|Name|Microbiote|Rearrangements|
|:-:|:-:|:-:|:-:|:-:|
|acetatifactor_muris|16||No|Duplication ?|
|akkermansia_muciniphila|9||Yes|Translocation phage|
|bacteroidetes_caecimuris|7||Yes|Two small translocations|
|bifidobacterium_animalis|14||No|Large inversion|
|blautia_coccoides|6||Yes|-|
|clostridium_clostrodioforme|10||Yes|Duplication ?|
|clostridium_innocuum|5||Yes|-|
|enterococcus_faecalis|11||No|-|
|flavonifractor_plautii|12||No|A lot of things|
|lactobacillus_reuteri|13||No|-|
|muribaculum_intestinale|8||Yes|-|
|trichuris_muris|15||No|-|

We generates the bam files in order to reorganize the genomes based on the reads and not the contact map.

In [1]:
%%script false --no-raise-error
%%bash 

# Script to align reads on genome (call alignement.sh later)

fasta=$1
forward=$2
reverse=$3
output=$4
threads=16

bwa index $fasta
bwa mem -5SP -t $threads $fasta $forward $reverse > $output.sam
samtools view -F 0x904 -bS $output.sam -@ $threads > $output.bam
samtools sort $output.bam -@ $threads > $output.sorted.bam
samtools index $output.sorted.bam -@ $threads

rm $output.sam $output.bam

## *Bifidobacterium animalis*

Align reads on genome 

```bash
/data/oligomm/alignment.sh bifidobacterium_animalis/B_animalis.fa \
    fastq/OMM10_rep_for.fastq.gz fastq/OMM10_rep_rev.fastq.gz \
    bifidobacterium_animalis/b_animalis -t 16
```

Zoom on the translocation with pyGenome tracks: 

```bash 
cd /data/oligomm/bifidobacterium_animalis/annotation
pyGenomeTracks --tracks b_animalis_tracks.ini --out zoom.pdf --region B_animalis:450000-550000
```

Find positions of inversion at 480,000 and 1,370,000 more or less 5kb. 

Use IGV with genes annotation the bam files and the reference genome to refine the position of the inversion.

Find positions for the inversion:

|start|end|orientation|
|:-:|:-:|:-:|
|1|483615|1|  
|483626|1371765|-1|  
|1371766|2021936|1|  

In [2]:
## Rewrite the fasta
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import pandas as pd


def write_fasta(
    init_fasta, info_frags, output, junction=False
):

    """Convert an info_frags.txt file into a fasta file given a reference.
    Optionally adds junction sequences to reflect the possibly missing base
    pairs between two newly joined scaffolds.
    """

    init_genome = {
        record.id: record.seq for record in SeqIO.parse(init_fasta, "fasta")
    }
    init_contig = None
    my_new_records = []
    with open(info_frags, "r") as info_frags_handle:
        current_seq = ""
        current_id = None
        previous_contig = None
        for line in info_frags_handle:
            if line.startswith(">"):
                previous_contig = None
                if current_id is not None:
                    new_record = SeqRecord(
                        current_seq, id=current_id, description=""
                    )
                    my_new_records.append(new_record)
                current_seq = ""
                current_id = str(line[1:])
            elif line.startswith("init_contig"):
                previous_contig = None
            else:
                (init_contig, _, orientation, pos_start, pos_end) = str(
                    line[:-1]
                ).split("\t")

                start = int(pos_start)
                end = int(pos_end)
                ori = int(orientation)

                assert start < end
                assert ori in {-1, 1}
                if junction and previous_contig not in {None, init_contig}:
                    error_was_raised = False
                    try:
                        extra_seq = Seq(junction)
                        current_seq += extra_seq
                    except TypeError:
                        if not error_was_raised:
                            print("Invalid junction sequence")
                            error_was_raised = True

                seq_to_add = init_genome[init_contig][start:end]
                if ori == 1:
                    current_seq += seq_to_add
                elif ori == -1:
                    current_seq += seq_to_add.reverse_complement()
                else:
                    raise ValueError("Invalid data in orientation field {}".format(ori))

                previous_contig = "A"

        new_record = SeqRecord(current_seq, id=current_id, description="")
        my_new_records.append(new_record)
    SeqIO.write(my_new_records, output, "fasta")

In [3]:
%%bash 

cd /data/oligomm/bifidobacterium_animalis/
cat new_info_frags.txt

>B_animalis
init_contig	id_frag	orientation	start	end
B_animalis	0	1	0	483616
B_animalis	0	-1	483626	1371766
B_animalis	0	1	1371766	2021936


In [4]:
import os

os.chdir("/data/oligomm/bifidobacterium_animalis/")
write_fasta("B_animalis.fa", "new_info_frags.txt", "B_animalis_v2.fa", junction="NNNNNNNNNN")

In [5]:
%%script false --no-raise-error
%%bash
# Verification by doing again the contact map with hictuff to look at the matrix.

hicstuff pipeline -C -d -DfF -g B_animalis_v2.fa -e DpnII,HinfI -pn -o 14_rep2_new_genome \
    -t 16 ../fastq/OMM10_rep_for.fastq.gz ../fastq/OMM10_rep_rev.fastq.gz
cd 14_rep2_new_genome
hicstuff view -n -b 500bp -f fragments_list.txt abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_500bp_new_genome.png -D 1000
hicstuff view -n -b 2kb -f fragments_list.txt abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_2kb_new_genome.png -D 1000

## *Flavonifractor plautii*

Align reads on genome 

```bash
/data/OligoMM/alignment.sh flavonifractor_plautii/F_plautii.fa \
    fastq/OMM8_rep_for.fastq.gz fastq/OMM8_rep_rev.fastq.gz \
    flavonifractor_plautii/F_plautii -t 16
```

Zoom on the translocation with pyGenome tracks: 

```bash
cd /data/OligoMM/flavonifractor_plautii/annotation
pyGenomeTracks --tracks f_plautii_tracks.ini --out zoom.pdf --region F_plautii:450000-550000
```

Find positions:
- duplication rRNA at 873,000 - 1,795,000 (let like that)
- inversion at 915,000 - 1,035,000
- translocation at 930,000-945,000 - 2,075,000 (not touch as no clear borders)
- translocation at 1,355,000 - 3,740,000-END 


Use IGV with genes annotation the bam files and the reference genome to refine the position of the inversion.

Find positions for the inversion:   
0 915342 1  
915352 1037195 -1  
1037205 1355964 1  
3743036 3813716 1  
1355974 3743046 1  

In [6]:
%%bash 
cat new_info_frags.txt

>B_animalis
init_contig	id_frag	orientation	start	end
B_animalis	0	1	0	483616
B_animalis	0	-1	483626	1371766
B_animalis	0	1	1371766	2021936


In [7]:
os.chdir("/data/oligomm/flavonifractor_plautii/")
write_fasta("F_plautii.fa", "new_info_frags.txt", "F_plautii_v2.fa", junction="NNNNNNNNNN")

In [8]:
%%script false --no-raise-error
%%bash
# Verification by doing again the contact map with hictuff to look at the matrix.

hicstuff pipeline -C -d -DfF -g F_plautii_v2.fa -e DpnII,HinfI -pn -o 12_rep2_new_genome \
    -t 16 ../fastq/OMM8_rep_for.fastq.gz ../fastq/OMM8_rep_rev.fastq.gz
cd 12_rep2_new_genome
hicstuff view -n -b 500bp -f fragments_list.txt abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_500bp_new_genome.png -D 1000
hicstuff view -n -b 2kb -f fragments_list.txt abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_2kb_new_genome.png -D 1000
hicstuff view -n -b 500bp -f ../12_rep2/fragments_list.txt ../12_rep2/abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_500bp_old_genome.png -D 1000
hicstuff view -n -b 2kb -f ../12_rep2/fragments_list.txt ../12_rep2/abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_2kb_old_genome.png -D 1000
    

## *Akkermansia muciniphila*

Align reads on genome 

```bash
/data/OligoMM/alignment.sh akkermansia_muciniphila/A_muciniphila.fa \
    fastq/OMM5_rep_for.fastq.gz fastq/OMM5_rep_rev.fastq.gz \
    akkermansia_muciniphila/A_muciniphila -t 16
```

Zoom on the translocation with pyGenome tracks: 

```bash
cd /data/OligoMM/akkermansia_muciniphila/annotation
pyGenomeTracks --tracks a_muciniphila_tracks.ini --out zoom.pdf --region A_muciniphila:450000-550000
```

Find positions:
- translocation at 2,305,000-2,320,000 -> 735,000


Use IGV with genes annotation the bam files and the reference genome to refine the position of the inversion.

Find positions for the inversion:     
0 733878 1    
2305800 2324871 -1  
733978  2305800 1   
2324871 2737358 1  


In [9]:
%%bash

cd ../akkermansia_muciniphila/
cat new_info_frags.txt

>A_muciniphila
init_contig	id_frag	orientation	start	end
A_muciniphila	0	1	0	733877
A_muciniphila	0	-1	2305800	2324870
A_muciniphila	0	1	733977	2305800
A_muciniphila	0	1	2324870	2737358


In [10]:
os.chdir("/data/oligomm/akkermansia_muciniphila/")
write_fasta("A_muciniphila.fa", "new_info_frags.txt", "A_muciniphila_v2.fa", junction="NNNNNNNNNN")

In [11]:
%%script false --no-raise-error
%%bash
# Verification by doing again the contact map with hictuff to look at the matrix.

hicstuff pipeline -C -d -DfF -g A_muciniphila_v2.fa -e DpnII,HinfI -pn -o 9_rep2_new_genome \
    -t 16 ../fastq/OMM5_rep_for.fastq.gz ../fastq/OMM5_rep_rev.fastq.gz
cd 9_rep2_new_genome
hicstuff view -n -b 500bp -f fragments_list.txt abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_500bp_new_genome.png -D 1000
hicstuff view -n -b 2kb -f fragments_list.txt abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_2kb_new_genome.png -D 1000
hicstuff view -n -b 500bp -f ../9_rep2/fragments_list.txt ../9_rep2/abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_500bp_old_genome.png -D 1000
hicstuff view -n -b 2kb -f ../9_rep2/fragments_list.txt ../9_rep2/abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_2kb_old_genome.png -D 1000
    

**It's duplicated**

## *Bacteroidetes caecimuris*

Align reads on genome 

```bash
/data/OligoMM/alignment.sh bacteroidetes_caecimuris/B_caecimuris.fa \
    fastq/OMM3_rep_for.fastq.gz fastq/OMM3_rep_rev.fastq.gz \
    bacteroidetes_caecimuris/B_caecimuris -t 16
```

Zoom on the translocation with pyGenome tracks: 

```bash
cd /data/OligoMM/bacteroidetes_caecimuris/annotation
pyGenomeTracks --tracks b_caecimuris_tracks.ini --out zoom.pdf --region B_caecimuris:450000-550000
```

Find positions:
- translocation at 4,205,000-4,220,000 -> 100,000
- translocation at 4,220,000-4,230,000 -> 1,685,000

Use IGV with genes annotation the bam files and the reference genome to refine the position of the inversion.

Find positions for the inversion:     
0 123230 1  
4209127 4219196 1  
123240  1685345 1   
4219206 4224320 1  
1685355 4209137 1  
4224330 4800606 1  

In [12]:
%%bash

cd ../bacteroidetes_caecimuris/
cat new_info_frags.txt

>B_caecimuris
init_contig	id_frag	orientation	start	end
B_caecimuris	0	1	0	123231
B_caecimuris	0	1	4209138	4219197
B_caecimuris	0	1	123241	1685346
B_caecimuris	0	1	4219207	4224321
B_caecimuris	0	1	1685356	4209128
B_caecimuris	0	1	4224331	4800607


In [13]:
os.chdir("/data/oligomm/bacteroidetes_caecimuris/")
write_fasta("B_caecimuris.fa", "new_info_frags.txt", "B_caecimuris_v2.fa", junction="NNNNNNNNNN")

In [14]:
%%script false --no-raise-error
%%bash
# Verification by doing again the contact map with hictuff to look at the matrix.

hicstuff pipeline -C -d -DfF -g B_caecimuris_v2.fa -e DpnII,HinfI -pn -o 7_rep2_new_genome \
    -t 16 ../fastq/OMM3_rep_for.fastq.gz ../fastq/OMM3_rep_rev.fastq.gz
cd 7_rep2_new_genome
hicstuff view -n -b 500bp -f fragments_list.txt abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_500bp_new_genome.png -D 1000
hicstuff view -n -b 2kb -f fragments_list.txt abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_2kb_new_genome.png -D 1000
hicstuff view -n -b 500bp -f ../7_rep2/fragments_list.txt ../7_rep2/abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_500bp_old_genome.png -D 1000
hicstuff view -n -b 2kb -f ../7_rep2/fragments_list.txt ../7_rep2/abs_fragments_contacts_weighted.txt \
    -T exp0.7 -o ../contact_map/in_vitro_R2_2kb_old_genome.png -D 1000
    